* GeoDaten https://gdz.bkg.bund.de/index.php/default/digitale-geodaten/verwaltungsgebiete/nuts-gebiete-1-5-000-000-stand-31-12-nuts5000-31-12.html

In [ ]:
#run(`wget --force-html -i https://opendata.dwd.de/climate_environment/CDC/derived_germany/techn/daily/bad_weather_days/recent/`)

In [67]:
using DataFrames, CSV,Statistics

In [ ]:
stations=CSV.File("data/stationslexikon.csv")|>DataFrame

In [83]:
df_count=DataFrame(count=Int64[],laenge=Float64[],breite=Float64[])
for file in readdir("data/")
    if !startswith(file,"swt_daten")
        continue
    end

    df=CSV.File(joinpath("data",file),delim=';')|>DataFrame;
    if size(df,1)==0
        continue
    end
    bad_days=size(df[(df."SWT-Behinderungsstufe".=="A").| (df."SWT-Behinderungsstufe".=="B"),:],1)
    
    station_place=stations[stations.Stations_ID.==df.STATIONS_ID[1],:]
    
    laenge=mean(station_place.Länge)
    breite=mean(station_place.Breite)
    df_count=vcat(df_count,DataFrame(count=bad_days,laenge=laenge,breite=breite))
end


In [22]:
using GeoMakie, GLMakie

In [27]:
using ArchGDAL

In [33]:
layer=ArchGDAL.getlayer(ArchGDAL.read("../nuts5000_12-31.gk3.shape/nuts5000/NUTS5000_N1.shp"),0)

Layer: NUTS5000_N1
  Geometry 0 (): [wkbPolygon], MULTIPOLYGON (((3477...), ...
     Field 0 (OBJID): [OFTString], DEBKGNU5000000C8, DEBKGNU5000000C9, ...
     Field 1 (BEGINN): [OFTDate], 2021-10-04T00:00:00, 2021-10-04T00:00:00, ...
     Field 2 (GF): [OFTInteger], 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ...
     Field 3 (NUTS_LEVEL): [OFTInteger], 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
     Field 4 (NUTS_CODE): [OFTString], DE1, DE2, DE3, DE4, DE5, DE6, DE7, ...
...
 Number of Fields: 6

In [50]:
bounds=DataFrame(layer);

In [51]:
ArchGDAL.createcoordtrans(ArchGDAL.importURL("https://sg.geodatenzentrum.de/web_public/gdz/dokumentation/crs/gk3.prj"), ArchGDAL.importPROJ4("+proj=longlat +datum=WGS84 +no_defs +type=crs")) do transform
    for x in eachrow(bounds)
    ArchGDAL.transform!(x."", transform)
    end
end

In [118]:
fig=Figure()
q=GridLayout(fig[1,1])
Label(q[1,1],text="Bad weather is not your best friend?", fontsize=30)
Label(q[2,1],text="Based on the bad weather-days in construction 2022-now")
ax=GeoAxis(fig[2,1],latlims=(46.9,55.1),lonlims=(5.5,15.5))
voronoiplot!(ax, df_count.laenge,df_count.breite,df_count.count,showgenerators=false,markersize=0,colormap=cgrad(:rainbow,alpha=0.4),colorrange=(0,60),highclip=:red)
s=scatter!(ax,df_count.laenge,df_count.breite,color=df_count.count,colormap=:rainbow, colorrange=(0,60),highclip=:red,markersize=10,marker='😄',strokecolor=:black)
poly!(ax,GeoMakie.to_multipoly.(GeoMakie.geo2basic.(bounds."")),color=:transparent,strokecolor=:black,strokewidth=4)
Colorbar(fig[2,2],s,vertical=true,label="Number of bad weather days with at least disability level difficult")
hidespines!(ax)
hidedecorations!(ax)
Label(fig[3,1],text="Data: © Deutscher Wetterdienst\nGeodata: © GeoBasis-DE / BKG (2023) ",justification=:left)

fig

In [119]:
save("bad.png",fig,px_per_unit=2)